In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from tkinter import Tk, filedialog
import sys
import pandas as pd
from matplotlib.animation import FuncAnimation

#  IA para Insuficiencia Cardíaca 

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import joblib
import warnings
warnings.filterwarnings('ignore')

#  Configuración inicial 
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12


# CLASE PREDICTORA (IA)

class HeartFailurePredictor:
    def __init__(self):
        self.scaler = StandardScaler()  # Primero creamos el scaler
        self.model = self.create_simulated_model()  # Luego el modelo
        
    def create_simulated_model(self):
        """Modelo basado en parámetros clínicos establecidos"""
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        
        # Datos simulados (reemplazar con dataset real)
        X_train = np.array([
            [-15.0, -18.0, -1.0, -1.2, 55, 125],  # Caso sano
            [-10.0, -12.0, -0.5, -0.6, 40, 300],   # Caso con IC
            [-14.5, -17.8, -0.9, -1.1, 52, 150],   # Caso sano
            [-9.0, -11.0, -0.4, -0.5, 38, 450]     # Caso con IC
        ])
        y_train = np.array([0, 1, 0, 1])  # 0: Sano, 1: IC
        
        self.scaler.fit(X_train)
        model.fit(self.scaler.transform(X_train), y_train)
        return model
    
    def predict(self, strain_data, clinical_data):
        """Predice probabilidad de IC"""
        X = np.array([[
            strain_data['Strain_Septal'],
            strain_data['Strain_Lateral'],
            strain_data['Strain_Rate_Septal'],
            strain_data['Strain_Rate_Lateral'],
            clinical_data['FEVI'],
            clinical_data['NT_proBNP']
        ]])
        return self.model.predict_proba(self.scaler.transform(X))[0, 1]


# ANÁLISIS DE VIDEO 

def select_video():
    """Interfaz para seleccionar archivo de video"""
    root = Tk()
    root.withdraw()
    video_path = filedialog.askopenfilename(
        title="Selecciona el video ecocardiográfico",
        filetypes=[("Videos", "*.avi *.mp4 *.mov"), ("Todos los archivos", "*.*")]
    )
    root.destroy()
    return video_path

# --- Cargar video ---
print("Seleccione el video:")
video_path = select_video()
if not video_path:
    print("No se seleccionó ningún archivo.")
    sys.exit()

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error al abrir el video. Verifica:")
    print(f"- Ruta: {video_path}")
    print("- Formato (convierte a MP4 si es necesario)")
    print("- Instala codecs con: pip install ffmpeg-python")
    sys.exit()

fps = cap.get(cv2.CAP_PROP_FPS)
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"\nVideo cargado: {video_path}")
print(f"Resolución: {int(cap.get(3))}x{int(cap.get(4))}")
print(f"FPS: {fps:.2f}, Total frames: {num_frames}")

# --- Selección manual de puntos ---
ret, frame1 = cap.read()
if not ret:
    print("Error al leer el primer frame.")
    sys.exit()

frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

plt.switch_backend('TkAgg')
plt.imshow(frame1_gray, cmap='gray')
plt.title("Seleccione 2 puntos:\n1. Pared septal (Click 1)\n2. Pared lateral (Click 2)")
print("\nPor favor, seleccione 2 puntos en la imagen...")
points = plt.ginput(2, timeout=0)
plt.close()

if len(points) < 2:
    print("Error: Debes seleccionar exactamente 2 puntos.")
    sys.exit()

x1, y1 = int(points[0][0]), int(points[0][1])
x2, y2 = int(points[1][0]), int(points[1][1])
roi_size = 10

print(f"\nPuntos seleccionados:")
print(f"- Pared septal: ({x1}, {y1})")
print(f"- Pared lateral: ({x2}, {y2})")

# --- Extraer parches de referencia ---
patch1 = frame1_gray[y1-roi_size:y1+roi_size, x1-roi_size:x1+roi_size]
patch2 = frame1_gray[y2-roi_size:y2+roi_size, x2-roi_size:x2+roi_size]

if patch1.size == 0 or patch2.size == 0:
    print("Error: Los puntos seleccionados están demasiado cerca del borde.")
    sys.exit()

# --- Procesamiento del video ---
displacements1, displacements2 = [y1], [y2]

print("\nIniciando procesamiento con visualización en tiempo real...")
print("Presione 'q' para salir de la visualización")

cv2.namedWindow('Seguimiento en Tiempo Real', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Seguimiento en Tiempo Real', 800, 600)

try:
    for i in range(1, num_frames):
        ret, frame = cap.read()
        if not ret:
            break
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Seguimiento por correlación cruzada
        c1 = cv2.matchTemplate(gray_frame, patch1, cv2.TM_CCOEFF_NORMED)
        c2 = cv2.matchTemplate(gray_frame, patch2, cv2.TM_CCOEFF_NORMED)
        
        _, max_val1, _, max_loc1 = cv2.minMaxLoc(c1)
        _, max_val2, _, max_loc2 = cv2.minMaxLoc(c2)
        
        if max_val1 > 0.5 and max_val2 > 0.5:
            y1_track = max_loc1[1] + roi_size
            y2_track = max_loc2[1] + roi_size
        else:
            y1_track = displacements1[-1]
            y2_track = displacements2[-1]
        
        displacements1.append(y1_track)
        displacements2.append(y2_track)
        
        # Visualización
        frame_display = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
        cv2.rectangle(frame_display, (x1-roi_size, y1-roi_size), (x1+roi_size, y1+roi_size), (255, 0, 0), 1)
        cv2.rectangle(frame_display, (x2-roi_size, y2-roi_size), (x2+roi_size, y2+roi_size), (0, 255, 0), 1)
        
        cross_size = 3
        cv2.line(frame_display, (x1-cross_size, int(y1_track)), (x1+cross_size, int(y1_track)), (0, 0, 255), 1)
        cv2.line(frame_display, (x1, int(y1_track)-cross_size), (x1, int(y1_track)+cross_size), (0, 0, 255), 1)
        cv2.line(frame_display, (x2-cross_size, int(y2_track)), (x2+cross_size, int(y2_track)), (0, 255, 0), 1)
        cv2.line(frame_display, (x2, int(y2_track)-cross_size), (x2, int(y2_track)+cross_size), (0, 255, 0), 1)
        
        cv2.imshow('Seguimiento en Tiempo Real', frame_display)
        if cv2.waitKey(int(1000/fps)) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("\nProcesamiento interrumpido por el usuario")
finally:
    cap.release()
    cv2.destroyAllWindows()

# --- Cálculo de deformación ---
displacements1 = np.array(displacements1)
displacements2 = np.array(displacements2)

defor1 = (displacements1 - displacements1[0]) / displacements1[0]
defor2 = (displacements2 - displacements2[0]) / displacements2[0]

dt = 1 / fps
strain_rate1 = np.diff(defor1) / dt
strain_rate2 = np.diff(defor2) / dt

# PREDICCIÓN CON IA 
def get_clinical_data():
    """Recoge datos clínicos adicionales"""
    print("\nINGRESE DATOS CLÍNICOS (valores por defecto entre paréntesis):")
    return {
        'FEVI': float(input("FEVI (%) (50): ") or 50),
        'NT_proBNP': float(input("NT-proBNP (pg/mL) (200): ") or 200),
        'Edad': int(input("Edad (60): ") or 60)
    }

# --- Resultados numéricos ---
print("\n=== RESULTADOS DEL ANÁLISIS ===")
print(f"Deformación máxima (Septal): {np.min(defor1):.4f}")
print(f"Deformación máxima (Lateral): {np.min(defor2):.4f}")
print(f"Strain Rate máximo (Septal): {np.min(strain_rate1):.2f} 1/s")
print(f"Strain Rate máximo (Lateral): {np.min(strain_rate2):.2f} 1/s")

# --- Predicción con IA ---
clinical_data = get_clinical_data()
predictor = HeartFailurePredictor()

strain_data = {
    'Strain_Septal': np.min(defor1) * 100,  # Convertir a porcentaje
    'Strain_Lateral': np.min(defor2) * 100,
    'Strain_Rate_Septal': np.min(strain_rate1),
    'Strain_Rate_Lateral': np.min(strain_rate2)
}

prob_ic = predictor.predict(strain_data, clinical_data)
risk_category = "ALTO RIESGO" if prob_ic >= 0.5 else "bajo riesgo"

# --- Visualización integrada ---
plt.figure(figsize=(16, 10))

# Gráfico 1: Puntos iniciales
plt.subplot(2, 2, 1)
plt.imshow(frame1_gray, cmap='gray')
plt.scatter([x1, x2], [y1, y2], c=['red', 'blue'], s=50)
plt.title("Puntos de seguimiento")

# Gráfico 2: Strain
plt.subplot(2, 2, 2)
plt.plot(defor1, 'r', label='Septal')
plt.plot(defor2, 'b', label='Lateral')
plt.title("Deformación (Strain)")
plt.legend()

# Gráfico 3: Strain Rate
plt.subplot(2, 2, 3)
plt.plot(strain_rate1, 'r', label='Septal Rate')
plt.plot(strain_rate2, 'b', label='Lateral Rate')
plt.title("Strain Rate (1/s)")

# Gráfico 4: Resultado IA
plt.subplot(2, 2, 4)

# Resultados del análisis
analysis_text = (
    "RESULTADOS DEL ANÁLISIS:\n"
    f"• Strain Septal: {np.min(defor1)*100:.1f}%\n"
    f"• Strain Lateral: {np.min(defor2)*100:.1f}%\n"
    f"• Strain Rate S: {np.min(strain_rate1):.1f} 1/s\n"
    f"• Strain Rate L: {np.min(strain_rate2):.1f} 1/s\n"
    
)

# Predicción de IC
prediction_text = (
    "\nPREDICCIÓN DE INSUFICIENCIA CARDÍACA:\n"
    f"Probabilidad: {prob_ic:.1%}\n"
    f"Riesgo: {risk_category}"
)

# Texto combinado
full_text = analysis_text + prediction_text

# Posiciones ajustadas para mejor visualización
plt.text(0.5, 0.90, "RESULTADOS CLÍNICOS", 
         ha='center', weight='bold', fontsize=12)
plt.text(0.5, 0.70, analysis_text, 
         ha='center', fontsize=10, va='top')
plt.text(0.5, 0.40, "PREDICCIÓN DE INSUFICIENCIA CARDÍACA", 
         ha='center', weight='bold', fontsize=12)
plt.text(0.5, 0.30, f"{prob_ic:.1%}", 
         ha='center', fontsize=24, color='red' if prob_ic >= 0.5 else 'green')
plt.text(0.5, 0.15, risk_category, 
         ha='center', fontsize=14, weight='bold')

plt.axis('off')
plt.tight_layout()
plt.show()

Seleccione el video:

Video cargado: C:/Users/geron/OneDrive/Escritorio/PYTHON/ESFUERZO/Eko/0X1A296F5FCD5A0ED8.avi
Resolución: 112x112
FPS: 50.00, Total frames: 131

Por favor, seleccione 2 puntos en la imagen...

Puntos seleccionados:
- Pared septal: (52, 70)
- Pared lateral: (92, 67)

Iniciando procesamiento con visualización en tiempo real...
Presione 'q' para salir de la visualización

=== RESULTADOS DEL ANÁLISIS ===
Deformación máxima (Septal): -0.1000
Deformación máxima (Lateral): -0.2090
Strain Rate máximo (Septal): -12.86 1/s
Strain Rate máximo (Lateral): -24.63 1/s

INGRESE DATOS CLÍNICOS (valores por defecto entre paréntesis):
